In [1]:
import pandas as pd
import json

In [2]:
from pymongo import MongoClient
myclient = MongoClient('mongodb://52.174.100.115:2001', username='itsvit', password='wj3as52tML4B7Z5c', authSource='admin')
mydb = myclient['main']
companies = mydb['companies']

In [3]:
# find the original sheet here: https://docs.google.com/spreadsheets/d/1-PLpBxsBurmPRhH6p78oyvl3kd70XjgDxQJaYKnQmzU/edit#gid=730097254

df = pd.read_excel(open('Tech radar.xlsx','rb'), sheet_name='sample_data')
rings = pd.read_excel(open('Tech radar.xlsx','rb'), sheet_name='stage-ring')

In [4]:
df.head()

,Name,Website,Focus field,Stage
0,Easby Electronics,easby.com,Advanced robotics,Observe
1,"hanneung electronics co.,ltd",hanneung.co.kr,Advanced robotics,Observe
2,Amantys,amantys.com,Advanced robotics,Observe
3,"Electronics Supply Co., Inc",eskc.com,Advanced robotics,Observe
4,LM Digital Co.,lmdigital.co.kr,Advanced robotics,Assess


In [5]:
company_ids = []
for url in df['Website'].values:
    try:
        curser = companies.find_one({'url':url}, {'_id': 1, 'url': 1})
        company_ids.append(str(curser['_id']))
    except:
        print(url)
        company_ids.append('')

In [6]:
# company_ids

In [7]:
df['company_ids'] = company_ids

In [8]:
rings = rings['stage'].tolist()

In [9]:
rings

['Adopt', 'Trial', 'Assess', 'Observe']

In [10]:
quadrant = df['Focus field'].unique().tolist()

In [11]:
quadrant

['Advanced robotics',
 'Artificial intelligence',
 'End-to-end connectivity',
 'Consistent planning',
 'Lifelong learning ',
 'Sustainable production',
 'Digital twin',
 'Integrated logistics']

In [12]:
entries = []
for index, row in df.iterrows():
#     if quadrant.index(row["Focus field"]) < 4:
        temp = {"link":row["company_ids"], "label":row["Name"], "quadrant":quadrant.index(row["Focus field"]), "ring":rings.index(row["Stage"]) , "active":True ,"moved":0}
        entries.append(temp)

In [13]:
# entries

In [14]:
color = ["#00C4B5", "#00A1FF", "#3679D5", "#222530"]

In [15]:
zolando_data = {
    "quadrants": [ {"name": cat} for cat in quadrant],
    "rings": [ {"name":ring, "color": c} for ring, c in zip(rings, color) ]
}

In [16]:
# zolando_data

In [17]:
data_tech_radar = {
    "quadrants": zolando_data['quadrants'],
    "rings": zolando_data['rings'],
    "entries": entries
}

In [18]:
data_tech_radar

{'quadrants': [{'name': 'Advanced robotics'},
  {'name': 'Artificial intelligence'},
  {'name': 'End-to-end connectivity'},
  {'name': 'Consistent planning'},
  {'name': 'Lifelong learning '},
  {'name': 'Sustainable production'},
  {'name': 'Digital twin'},
  {'name': 'Integrated logistics'}],
 'rings': [{'name': 'Adopt', 'color': '#00C4B5'},
  {'name': 'Trial', 'color': '#00A1FF'},
  {'name': 'Assess', 'color': '#3679D5'},
  {'name': 'Observe', 'color': '#222530'}],
 'entries': [{'link': '5b23bccab6fa1e2f73019f29',
   'label': 'Easby Electronics',
   'quadrant': 0,
   'ring': 3,
   'active': True,
   'moved': 0},
  {'link': '5c7e809e3807d8150ff4b92f',
   'label': 'hanneung electronics co.,ltd',
   'quadrant': 0,
   'ring': 3,
   'active': True,
   'moved': 0},
  {'link': '5b1aaef372cc30742ea1b258',
   'label': 'Amantys',
   'quadrant': 0,
   'ring': 3,
   'active': True,
   'moved': 0},
  {'link': '6226423d5651d9d6fc3b2287',
   'label': 'Electronics Supply Co., Inc',
   'quadrant': 0

In [19]:
import json
with open('../data/data_tech_radar.json', 'w', encoding='utf-8') as f:
    json.dump(data_tech_radar, f)